In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets,transforms,models
import matplotlib.pyplot as plt
import time
import os
import copy
import torchmetrics
from torchmetrics.classification import Accuracy, Precision, Recall, F1Score

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
mean = np.array([0.485,0.456,0.406])
std = np.array([0.229,0.224,0.225])

data_transforms = {
    'train':transforms.Compose([
        transforms.Resize((30, 30)),
        transforms.ToTensor(),
        # transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # Repeat the single channel grayscale image three times to create an RGB image
        transforms.Normalize(mean, std)  # Normalize all three channels
    ]),
    'val':transforms.Compose([
        transforms.Resize((30, 30)),
        transforms.ToTensor(),
        # transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # Repeat the single channel grayscale image three times to create an RGB image
        transforms.Normalize(mean, std)  # Normalize all three channels
    ])
}

In [4]:
data_dir = 'train_dataset'
sets = ['train','val']
image_datasets = {x:datasets.ImageFolder(os.path.join(data_dir,x),
                                         data_transforms[x]) 
                  for x in ['train','val']}
dataloaders = {x:torch.utils.data.DataLoader(image_datasets[x],batch_size=4,
                                             shuffle=True,num_workers=4)
                for x in ['train','val']}


In [5]:
dataset_sizes = {x:len(image_datasets[x]) for x in ['train','val']}
class_names = image_datasets['train'].classes
print(class_names)

['1', '2', '3']


In [6]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    # initialize metric
    metric = torchmetrics.classification.Accuracy(task="multiclass", num_classes=3)
    predicted_labels = []
    ground_truth_labels = []
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-'*10)
        
        #Training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            #Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                #forward
                #track history only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _,preds = torch.max(outputs,1)
                    loss = criterion(outputs,labels)
                    
                    #backward + optimize only in train
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                
                #statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            predicted_labels.append(preds.cpu())
            ground_truth_labels.append(labels.cpu())

            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            #deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        print()
    
    
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')
    
    #calculate accuracy
    predicted_labels = torch.cat(predicted_labels)
    ground_truth_labels = torch.cat(ground_truth_labels)
    accuracy = Accuracy(task="multiclass", num_classes=3)
    accuracy(predicted_labels, ground_truth_labels)
    print(f'Accuracy: {accuracy.compute():.4f}')
    
    #calculate precision
    precision = Precision(task="multiclass", average='macro', num_classes=3)
    precision(predicted_labels, ground_truth_labels)
    print(f'Precision: {precision.compute():.4f}')
    
    #calculate recall
    recall = Recall(task="multiclass", average='macro', num_classes=3)
    recall(predicted_labels, ground_truth_labels)
    print(f'Recall: {recall.compute():.4f}')
    
    #calculate f1 score
    f1 = F1Score(task="multiclass", average='macro', num_classes=3)
    f1(predicted_labels, ground_truth_labels)
    print(f'F1: {f1.compute():.4f}')
    
    #calculate confusion matrix
    cm = torchmetrics.functional.confusion_matrix(predicted_labels, ground_truth_labels, num_classes=3, task="multiclass")
    print(f'Confusion Matrix: \n{cm}')    
    #load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [7]:
#cnn class
class ConvNet(nn.Module):
    def __init__(self, num_classes=3):
            super(ConvNet, self).__init__()
            
            #Output size after convolution filter
            #((w-f+2P)/s) +1 where w=width, f=filter, p=padding, s=stride
            
            #Input shape = (4, 1, 30, 30) (batch_size, channels, height, width)
            
            self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=1, stride=1, padding=1)

            #Shape= (4, 12, 30, 30)
            self.bn1 = nn.BatchNorm2d(num_features=12)
            #Shape= (4, 12, 30, 30)
            self.relu1 = nn.ReLU()
            #Shape= (4, 12, 30, 30)
            
            self.pool = nn.MaxPool2d(kernel_size=1)
            #Reduce the image size be factor 1, basically stays the same
            #Shape= (4, 12, 30, 30)
            
            
            self.conv2 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=1, stride=1, padding=1)
            #Shape= (4, 20, 30, 30)
            self.relu2 = nn.ReLU()
            #Shape= (4, 20, 30, 30)
            
            
            self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=1, stride=1, padding=1)
            #Shape= (4, 32, 30, 30)
            self.bn3 = nn.BatchNorm2d(num_features=32)
            #Shape= (4, 32, 30, 30)
            self.relu3 = nn.ReLU()
            #Shape= (4, 32, 30, 30)
            
            self.fc = nn.Linear(in_features=32*36*36, out_features=num_classes)
            
            
    #Feed forwad function
            
    def forward(self, input):
            output = self.conv1(input)
            output = self.bn1(output)
            output = self.relu1(output)
            
            output = self.pool(output)
            
            output = self.conv2(output)
            output = self.relu2(output)
            
            output = self.conv3(output)
            output = self.bn3(output)
            output = self.relu3(output)
            
            output = output.view(-1, 32*36*36)
            
            output = self.fc(output)
            
            return output

In [8]:
#load the best_checkpoint.model
checkpoint = torch.load('best_checkpoint.model')
pretrained_model = ConvNet(num_classes=3)
pretrained_model.load_state_dict(checkpoint)

for param in pretrained_model.parameters():
    param.requires_grad = False

#exchange the last layer
num_ftrs = pretrained_model.fc.in_features

pretrained_model.fc = nn.Linear(num_ftrs, 3)
pretrained_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pretrained_model.parameters(),lr=0.001)

#scheduler
step_lr_scheduler = lr_scheduler.StepLR(optimizer,step_size=7,gamma=0.1)

model_ft = train_model(pretrained_model,criterion,optimizer,step_lr_scheduler,num_epochs=25)

RuntimeError: Error(s) in loading state_dict for ConvNet:
	size mismatch for conv1.weight: copying a param with shape torch.Size([12, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([12, 3, 1, 1]).
	size mismatch for conv2.weight: copying a param with shape torch.Size([20, 12, 3, 3]) from checkpoint, the shape in current model is torch.Size([20, 12, 1, 1]).
	size mismatch for conv3.weight: copying a param with shape torch.Size([32, 20, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 20, 1, 1]).
	size mismatch for fc.weight: copying a param with shape torch.Size([3, 7200]) from checkpoint, the shape in current model is torch.Size([3, 41472]).